In [1]:
import sys;  sys.path.append('..')

In [2]:
from shape_transformers.dataset.nphm_dataset import NPHMDataset
from pathlib import Path

data_path = Path('/apollo/datasets/NPHM/')
ds_train = NPHMDataset(data_path, subset='train')

In [3]:
from torch.utils.data import DataLoader

In [4]:
batch_size = 128

dl_train = DataLoader(
    ds_train,
    batch_size=batch_size,
    num_workers=8
)

In [5]:
def agg_means(means, batch_size, last_batch_size):
    return (
        torch.sum(means[:-1, ...] * batch_size / len(ds_train), dim=0)
        + (last_batch_size / len(ds_train)) * means[-1, ...]
    )

In [6]:
from tqdm.notebook import tqdm
import torch

v_means = []

for verts, faces, ids in tqdm(dl_train):
    verts = verts.to('cuda')
    v_means.append(
        verts.mean(dim=0)
    )
    last_batch_size = len(verts)

v_means = torch.stack(v_means)
v_mean = agg_means(v_means, batch_size, last_batch_size)

  0%|          | 0/41 [00:00<?, ?it/s]

In [7]:
v_vars = []

for verts, faces, ids in tqdm(dl_train):
    verts = verts.to('cuda')
    v_vars.append(
        (verts - v_mean).pow(2).mean(dim=0)
    )
    last_batch_size = len(verts)

v_vars = torch.stack(v_vars)
v_std = torch.sqrt(agg_means(v_vars, batch_size, last_batch_size))

  0%|          | 0/41 [00:00<?, ?it/s]

In [8]:
v_mean = v_mean.cpu().numpy()
v_std = v_std.cpu().numpy()

In [9]:
import numpy as np

np.save('../shape_transformers/dataset/nphm_mean_vertices.npy', v_mean)
np.save('../shape_transformers/dataset/nphm_std_vertices.npy', v_std)